Vorgehensweise für das Area 51-Special

Den Datensatz einlesen

Wir laden den UFO-Datensatz mit Pandas.

Prüfen die Spalten und Datenstruktur, um sicherzustellen, dass die relevanten Informationen enthalten sind.

Area 51-Koordinaten bestimmen

Area 51 liegt in Nevada, USA. Wir brauchen den genauen geografischen Bereich (Longitude & Latitude).

Daten filtern

Wir filtern nur Sichtungen, die innerhalb des Koordinatenbereichs von Area 51 liegen.

Analysen durchführen

Anzahl der Sichtungen berechnen.

Tageszeit der Sichtungen analysieren.

Formen der Objekte auswerten.

Trends visualisieren (z. B. Histogramm für die Tageszeiten).

Ergebnisse visualisieren

Diagramme erstellen (z. B. Barplot, Lineplot oder Scatterplot)

In [54]:
import pandas as pd

In [55]:
df_area51 = pd.read_csv(r"C:\Users\Admin\Desktop\Data Craft\UFO\Projekt_UFO\data\data_clean\ufo_sightings_scrubbed_clean.csv")

In [56]:
df_area51.head()

,Unnamed: 0,datetime,city,state,country,shape,duration_seconds,duration_hour_min,comments,date_posted,latitude,longitude
0,0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111
1,1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082
2,2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667
3,3,1956-10-10 21:00:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833
4,4,1960-10-10 20:00:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611


In [57]:
df_area51.columns

Index(['Unnamed: 0', 'datetime', 'city', 'state', 'country', 'shape',
       'duration_seconds', 'duration_hour_min', 'comments', 'date_posted',
       'latitude', 'longitude'],
      dtype='object')

In [58]:
df_area51.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         80332 non-null  int64  
 1   datetime           80332 non-null  object 
 2   city               80332 non-null  object 
 3   state              74535 non-null  object 
 4   country            70662 non-null  object 
 5   shape              78400 non-null  object 
 6   duration_seconds   80332 non-null  int64  
 7   duration_hour_min  80332 non-null  object 
 8   comments           80317 non-null  object 
 9   date_posted        80332 non-null  object 
 10  latitude           80332 non-null  float64
 11  longitude          80332 non-null  float64
dtypes: float64(2), int64(2), object(8)
memory usage: 7.4+ MB


In [59]:
df_area51.isnull().sum()

Unnamed: 0              0
datetime                0
city                    0
state                5797
country              9670
shape                1932
duration_seconds        0
duration_hour_min       0
comments               15
date_posted             0
latitude                0
longitude               0
dtype: int64

Area 51 liegt in Nevada, USA, nahe der Groom Lake Air Force Base .
Die ungefähren Koordinaten von Area 51 sind:

Wir filtern jetzt alle Sichtungen heraus, die innerhalb dieses Bereich sind
Geocoder für Area 51


In [60]:
# pip install geopy
# Importiere die Nominatim-Klasse aus der geopy.geocoders-Bibliothek.
# Nominatim ist ein Geocoding-Service, der Adressen in geografische Koordinaten umwandelt.

from geopy.geocoders import Nominatim

# Erstelle ein Geolokalisierungsobjekt (geolokalisierer).
# Der Parameter "user_agent" ist notwendig, da Nominatim diesen benötigt, um den Ursprung der Anfrage zu identifizieren.

geolokalisierer = Nominatim(user_agent="ufo_sightings_analysis")

# Verwende die Methode "geocode", um die Adresse "Area 51, Nevada, USA" in geografische Koordinaten (Breitengrad und Längengrad) umzuwandeln.
# Diese Methode sendet eine Anfrage an den Nominatim-Service.
standort = geolokalisierer.geocode("Area 51, Nevada, USA")

# Überprüfe, ob der Standort gefunden wurde.
if standort:
    # Falls ein Standort gefunden wurde, gib den Breitengrad (latitude) und Längengrad (longitude) aus.
    # f-Strings werden verwendet, um die Variablen direkt in den Ausgabestring einzubetten.
    print(f"Breitengrad: {standort.latitude}, Längengrad: {standort.longitude}")
else:
    # Falls kein Standort gefunden wurde, informiere den Benutzer darüber.
    print("Standort nicht gefunden.")


Breitengrad: 37.2360646, Längengrad: -115.812175


In [61]:
#  Koordinaten von Area 51
area51_lat = 37.2360646
area51_lon = -115.812175


radius = 0.05#entspricht 5 kilometer

df_area51 = pd.read_csv(r"C:\Users\Admin\Desktop\Data Craft\UFO\Projekt_UFO\data\data_clean\ufo_sightings_scrubbed_clean.csv")

# Filtern der Sichtungen innerhalb des definierten Radius
#In Pandas wird der &-Operator verwendet, um mehrere Bedingungen für das Filtern eines DataFrames zu kombinieren,
#da er sicherstellt, dass beide Bedingungen gleichzeitig wahr sind, während ein Komma zu einem Fehler führen würde,
#  weil es die Bedingungen als separate Argumente interpretiert
df_area51 = df_area51[  # Verwende den richtigen DataFrame "df_area51"
    (df_area51["latitude"].astype(float) >= area51_lat - radius) & 
    (df_area51["latitude"].astype(float) <= area51_lat + radius) & 
    (df_area51["longitude"].astype(float) >= area51_lon - radius) & 
    (df_area51["longitude"].astype(float) <= area51_lon + radius)
]

# Ergebnisse anzeigen
print(f"Anzahl der Sichtungen in der Nähe von Area 51: {len(df_area51)}")
print(df_area51.head())

Anzahl der Sichtungen in der Nähe von Area 51: 4
       Unnamed: 0             datetime               city state country  \
5057         5057  2004-10-28 20:25:00  area 51 (west of)    nv     NaN   
13944       13944  1999-11-03 02:00:00            area 51    nv     NaN   
14683       14683  1989-11-05 02:00:00         groom lake    nv     NaN   
48985       48985  1992-06-15 18:00:00            area 51    nv     NaN   

          shape  duration_seconds duration_hour_min  \
5057   fireball                10        10 seconds   
13944    circle              3600        app 1 hour   
14683       egg                12            12secs   
48985      oval             43200           12 hrs.   

                                                comments date_posted  \
5057          Two fire balls entering earths atmosphere.  2013-12-02   
13944  My brother and I witnessed a bright pulsating ...  2000-02-16   
14683  This is a little long&#44 so please hang in th...  1999-11-30   
48985  4 ad

In [62]:
radius10 = 0.10
df_area51 = df_area51[  # Verwende den richtigen DataFrame "df_area51"
    (df_area51["latitude"].astype(float) >= area51_lat - radius) & 
    (df_area51["latitude"].astype(float) <= area51_lat + radius) & 
    (df_area51["longitude"].astype(float) >= area51_lon - radius) & 
    (df_area51["longitude"].astype(float) <= area51_lon + radius)
]

# Ergebnisse anzeigen
print(f"Anzahl der Sichtungen in der Nähe von Area 51: {len(df_area51)}")
print(df_area51.head())

Anzahl der Sichtungen in der Nähe von Area 51: 4
       Unnamed: 0             datetime               city state country  \
5057         5057  2004-10-28 20:25:00  area 51 (west of)    nv     NaN   
13944       13944  1999-11-03 02:00:00            area 51    nv     NaN   
14683       14683  1989-11-05 02:00:00         groom lake    nv     NaN   
48985       48985  1992-06-15 18:00:00            area 51    nv     NaN   

          shape  duration_seconds duration_hour_min  \
5057   fireball                10        10 seconds   
13944    circle              3600        app 1 hour   
14683       egg                12            12secs   
48985      oval             43200           12 hrs.   

                                                comments date_posted  \
5057          Two fire balls entering earths atmosphere.  2013-12-02   
13944  My brother and I witnessed a bright pulsating ...  2000-02-16   
14683  This is a little long&#44 so please hang in th...  1999-11-30   
48985  4 ad

In [63]:
radius20 = 0.20
df_area51 = df_area51[  # Verwende den richtigen DataFrame "df_area51"
    (df_area51["latitude"].astype(float) >= area51_lat - radius) & 
    (df_area51["latitude"].astype(float) <= area51_lat + radius) & 
    (df_area51["longitude"].astype(float) >= area51_lon - radius) & 
    (df_area51["longitude"].astype(float) <= area51_lon + radius)
]

# Ergebnisse anzeigen
print(f"Anzahl der Sichtungen in der Nähe von Area 51: {len(df_area51)}")
print(df_area51.head())

Anzahl der Sichtungen in der Nähe von Area 51: 4
       Unnamed: 0             datetime               city state country  \
5057         5057  2004-10-28 20:25:00  area 51 (west of)    nv     NaN   
13944       13944  1999-11-03 02:00:00            area 51    nv     NaN   
14683       14683  1989-11-05 02:00:00         groom lake    nv     NaN   
48985       48985  1992-06-15 18:00:00            area 51    nv     NaN   

          shape  duration_seconds duration_hour_min  \
5057   fireball                10        10 seconds   
13944    circle              3600        app 1 hour   
14683       egg                12            12secs   
48985      oval             43200           12 hrs.   

                                                comments date_posted  \
5057          Two fire balls entering earths atmosphere.  2013-12-02   
13944  My brother and I witnessed a bright pulsating ...  2000-02-16   
14683  This is a little long&#44 so please hang in th...  1999-11-30   
48985  4 ad

In [64]:
radius30 = 30
df_area51 = df_area51[  # Verwende den richtigen DataFrame "df_area51"
    (df_area51["latitude"].astype(float) >= area51_lat - radius) & 
    (df_area51["latitude"].astype(float) <= area51_lat + radius) & 
    (df_area51["longitude"].astype(float) >= area51_lon - radius) & 
    (df_area51["longitude"].astype(float) <= area51_lon + radius)
]

# Ergebnisse anzeigen
print(f"Anzahl der Sichtungen in der Nähe von Area 51: {len(df_area51)}")
print(df_area51.head())

Anzahl der Sichtungen in der Nähe von Area 51: 4
       Unnamed: 0             datetime               city state country  \
5057         5057  2004-10-28 20:25:00  area 51 (west of)    nv     NaN   
13944       13944  1999-11-03 02:00:00            area 51    nv     NaN   
14683       14683  1989-11-05 02:00:00         groom lake    nv     NaN   
48985       48985  1992-06-15 18:00:00            area 51    nv     NaN   

          shape  duration_seconds duration_hour_min  \
5057   fireball                10        10 seconds   
13944    circle              3600        app 1 hour   
14683       egg                12            12secs   
48985      oval             43200           12 hrs.   

                                                comments date_posted  \
5057          Two fire balls entering earths atmosphere.  2013-12-02   
13944  My brother and I witnessed a bright pulsating ...  2000-02-16   
14683  This is a little long&#44 so please hang in th...  1999-11-30   
48985  4 ad

Sichtungen nach Tageszeit analysieren

In [65]:
# wandeln die 'duration_seconds' in ein timedelta-Objekt um
df_area51["duration"] = pd.to_timedelta(df_area51["duration_seconds"], unit= "s")
print(df_area51["duration"].head())

5057    0 days 00:00:10
13944   0 days 01:00:00
14683   0 days 00:00:12
48985   0 days 12:00:00
Name: duration, dtype: timedelta64[ns]


In [66]:
df_area51["duration_hour"] = df_area51["duration"].dt.total_seconds() / 3600
print(df_area51[["duration_seconds", "duration", "duration_hour" ]].head())

       duration_seconds        duration  duration_hour
5057                 10 0 days 00:00:10       0.002778
13944              3600 0 days 01:00:00       1.000000
14683                12 0 days 00:00:12       0.003333
48985             43200 0 days 12:00:00      12.000000


In [71]:

df_area51["time_of_day"] = df_area51["duration_hour"].apply(lambda x: "Tag" if 6 <= x < 18 else "Nacht" )
print(df_area51[["datetime", "duration_hour", "time_of_day"]].head())

                  datetime  duration_hour time_of_day
5057   2004-10-28 20:25:00       0.002778       Nacht
13944  1999-11-03 02:00:00       1.000000       Nacht
14683  1989-11-05 02:00:00       0.003333       Nacht
48985  1992-06-15 18:00:00      12.000000         Tag


In [81]:
df_area51

,Unnamed: 0,datetime,city,state,country,shape,duration_seconds,duration_hour_min,comments,date_posted,latitude,longitude,duration,duration_hour,time_of_day,duration_category,hour
5057,5057,2004-10-28 20:25:00,area 51 (west of),nv,NaN,fireball,10,10 seconds,Two fire balls entering earths atmosphere.,2013-12-02,37.252526,-115.800413,0 days 00:00:10,0.002778,Nacht,Kurz,20
13944,13944,1999-11-03 02:00:00,area 51,nv,NaN,circle,3600,app 1 hour,My brother and I witnessed a bright pulsating ...,2000-02-16,37.252526,-115.800413,0 days 01:00:00,1.000000,Nacht,Lang,2
14683,14683,1989-11-05 02:00:00,groom lake,nv,NaN,egg,12,12secs,This is a little long&#44 so please hang in th...,1999-11-30,37.276675,-115.798936,0 days 00:00:12,0.003333,Nacht,Kurz,2
48985,48985,1992-06-15 18:00:00,area 51,nv,NaN,oval,43200,12 hrs.,4 adults video tape object over area 51 in Nev...,1998-03-07,37.252526,-115.800413,0 days 12:00:00,12.000000,Nacht,Lang,18


In [ ]:
# Stelle sicher, dass 'datetime' ein DateTime-Objekt ist
df_area51['datetime'] = pd.to_datetime(df_area51['datetime'], errors='ignore')

# Extrahiere die Stunde der Sichtung
df_area51['hour'] = df_area51['datetime'].dt.hour

# Weise die Tageszeit basierend auf der Sichtungsstunde zu
df_area51["time_of_day"] = df_area51["hour"].apply(lambda x: "Tag" if 6 <= x < 18 else "Nacht")

# Überprüfe das Ergebnis
print(df_area51[["datetime", "duration_seconds" ,"time_of_day"]].head())


                 datetime  hour time_of_day
5057  2004-10-28 20:25:00    20       Nacht
13944 1999-11-03 02:00:00     2       Nacht
14683 1989-11-05 02:00:00     2       Nacht
48985 1992-06-15 18:00:00    18       Nacht


C:\Users\Admin\AppData\Local\Temp\ipykernel_6288\3591551528.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_area51['datetime'] = pd.to_datetime(df_area51['datetime'], errors='ignore')
